# Solving bubble dynamics with the Gilmore-Akulichev model

This notebook provides an example for simulating bubble dynamics and calculating the radiated pressure using the BubbleDynamics class.

In [ ]:
import sys

sys.path.append("..")
from wavelet_transform.utils.bubbledynamics import BubbleDynamics as BD
import numpy as np
import plotly.express as px
from IPython.display import display

Define the sonication parameters and the initial bubble radius and velocity. By default, we consider a vapour bubble in an unbounded aqueous medium at room temperature (20 C). The sonication signal is considered to be a monotone signal as follows: $~p_{ac} \times sin(2 \pi f t)$. 

In [ ]:
pac = 1e6  # the amplitude of the sine wave in Pa
f0_frequency = 1.1* 1e6  # insonation frequency (Hz)
period = 1 / f0_frequency # the period of the acoustic wave at the fundamental frequency 
no_cycle = 10  # no. of cycles of the sine wave
fs = f0_frequency * 1e3  # sampling rate to generate the time vector

time_delay = 0#4 * period  # time delay, t1
pulse_width = 10 * no_cycle * period  # pulse width, W = 1 / Q
phase_initial = 0#-0.325 * period  # tau
phase_shift = 0#np.pi / 4  # phi
no_harmonics = 1   # k


deltat = 1.0 / fs
tmax = no_cycle * period
time = np.arange(
    0.0, tmax, deltat
)  # time vector for computing the sine wave, in seconds


# R_init = 0.523e-6  # initial bubble radius in meters
R_init = 30e-6  # initial bubble radius in meters
Rdot_init = 0  # initial bubble's wall velocity in m/s

r_measure = 1e-4 # distance from the centre of bubble which radiated pressure (acoustic emission from bubble) is calculated over time

Setting up the bubble dynamics object for the above parameters.

In [ ]:
BD_obj = BD(
    R_init=R_init,
    Rdot_init = Rdot_init,
    frequency = f0_frequency,
    pac = pac,
    time_delay = time_delay,
    pulse_width = pulse_width,
    phase_initial = phase_initial,
    phase_shift = phase_shift,
    no_harmonics = no_harmonics,
    time = time,
    deltat = deltat,
)

In [ ]:
BD_obj.initialize()

In [ ]:
print("The resonance frequency of this bubble is: {0:.4f} MHz,\nIt is {1:.4f} times of the excitation frequency.".format(
    BD_obj.resonance_frequency * 1e-6, BD_obj.resonance_frequency/f0_frequency))

In [ ]:
print("The resonance period of this bubble is: {0:.4f} times of the excitation period".format(1/BD_obj.resonance_frequency / period))

In [ ]:
fig = px.line(
    x=BD_obj.time / period, y=BD_obj.npac_wave * BD_obj.medium_parameters["p0"] / 1e6
)
fig.update_layout(
    font_family="Arial",
    xaxis=dict(title="time in periods [t/T]"),
    yaxis=dict(title="acoustic pressure [MPa]"),
    title="Incident sound wave"
)
fig.show()

Solve the bubble dynamics modelled by the nonlinear Gilmore ordinary differential equation

In [ ]:
BD_obj.solver()

Visualise the results:

In [ ]:
fig = px.line(x=BD_obj.time / period, y=BD_obj.R)
fig.update_layout(
    xaxis=dict(title="time in periods [t/T]"),
    yaxis=dict(title="R / R<sub>0</sub> [DL]"), font_size=16,
    title="Bubble radius vs time"
)
fig.show()

In [ ]:
fig = px.line(x=BD_obj.time, y=BD_obj.Rdot)
fig.update_layout(
    font_family="'Serif",
    xaxis=dict(title="time [s]"),
    yaxis=dict(title="Relative <span>&#7768;</span> [ <span>&#7768;</span> /R<sub>0</sub> &#969;]"),
    title="Bubble radius velocity vs time",
    font_size=12
)
fig.show()

# The HTML coding is used for greek letters and sub/superscripts and overdots in the labels. 
# The list of unicodes of letters can be found here: https://www.compart.com/en/unicode/U+03C9

Now, we can calculate the emitted acoustic wave (measured irradiated pressure) from this oscillating bubble at a distance `r_measure` from the centre of the bubble, as follows

In [ ]:
BD_obj.calculate_p_radiated_vokurka(r_measure=r_measure)
BD_obj.calculate_p_radiated(r_measure=r_measure)

In [ ]:
import pandas as pd

rad_p_df1 = pd.DataFrame(
    data={
        "x": BD_obj.time,
        "y": BD_obj.p_radiated_vokurka,
        "name": "p_radiated_vokurka",
    }
)
rad_p_df2 = pd.DataFrame(
    data={
        "x": BD_obj.time,
        "y": BD_obj.p_radiated,
        "name": "p_radiated_akulichev",
    }
)
rad_p_df = pd.concat([rad_p_df1])  #, rad_p_df2])

fig = px.line(rad_p_df,x="x", y="y", color="name")

fig.update_layout(
    xaxis=dict(title="time [s]"),
    yaxis=dict(title="Relative radiated pressure [p / p<sub>0</sub>]"), #r"$\text{Relative radiated pressure } [p/p_0]$"),
)
fig.show()


In [ ]:
import wavelet_transform.utils.statistical_features as st
import scipy.signal as ss


In [ ]:
sig = ss.detrend(BD_obj.p_radiated_vokurka).copy()

In [ ]:
fft_amp, fft_freq = st.psd(sig, sampling_rate=fs, fundamental_freq=f0_frequency)

In [ ]:
freq_lim = 20
freq_index = fft_freq<freq_lim
fig_new = px.line(x=fft_freq[freq_index], y=10*np.log10(fft_amp[freq_index]))
fig_new.update_layout(
    yaxis_range=[-40,70],
    xaxis=dict(title="Freq [F/F<sub>0</sub>]"),
    yaxis=dict(title=r"PSD [dB]"), font_size=16
)
fig_new.show()

Plotting the dynamic phase (R - Rdot plot). The phase space plot can be used to classify the dynamics of cavitation.

In [ ]:
from scipy.stats import cauchy, laplace, norm, chi2
ch_pdf_data = cauchy.rvs(scale=1,size=100000)
lap_pdf_data = laplace.rvs(size=100000)
gaus_pdf_data = norm.rvs(scale=1,size=100000)
chis_pdf_data = chi2.rvs(df=2,size=100000)

In [ ]:
print('Gaussian pdf data: \n \t crest factor: {0}, \n \t kurtosis: {1}, \n \t spectral entropy: {2}, \n \t entropy: {3}, \n \t average energy: {4}'.format(
    st.crest_factor(gaus_pdf_data), st.kurtosis(gaus_pdf_data),st.spectral_entropy(gaus_pdf_data),st.entropy(gaus_pdf_data), st.avg_energy(gaus_pdf_data)))

print('Laplace pdf data: \n \t crest factor: {0}, \n \t kurtosis: {1}, \n \t spectral entropy: {2}, \n \t entropy: {3}, \n \t average energy: {4}'.format(
    st.crest_factor(lap_pdf_data), st.kurtosis(lap_pdf_data),st.spectral_entropy(lap_pdf_data),st.entropy(lap_pdf_data), st.avg_energy(lap_pdf_data)))

print('Chisq pdf data: \n \t crest factor: {0}, \n \t kurtosis: {1}, \n \t spectral entropy: {2}, \n \t entropy: {3}, \n \t average energy: {4}'.format(
    st.crest_factor(chis_pdf_data), st.kurtosis(chis_pdf_data),st.spectral_entropy(chis_pdf_data),st.entropy(chis_pdf_data), st.avg_energy(chis_pdf_data)))

print('Cauchy pdf data: \n \t crest factor: {0}, \n \t kurtosis: {1}, \n \t spectral entropy: {2}, \n \t entropy: {3}, \n \t average energy: {4}'.format(
    st.crest_factor(ch_pdf_data), st.kurtosis(ch_pdf_data),st.spectral_entropy(ch_pdf_data),st.entropy(ch_pdf_data), st.avg_energy(ch_pdf_data)))

